%conda install graphviz
!pip install --upgrade pip

In [ ]:
from diagrams import Cluster, Diagram, Edge
from diagrams.aws.compute import ECS
from diagrams.aws.ml  import Sagemaker, SagemakerModel, SagemakerNotebook, SagemakerTrainingJob
from diagrams.onprem.ci import Jenkins
from diagrams.onprem.iac import Awx, Ansible

graph_attr = {
    "fontsize": "60",
    "compund": "True",
    "splines":"spline",
}
cluster_attr = {
    "fontsize": "40",
}
node_attr = {
    "fontsize": "20",
    "height": "10.6",
}
edge_attr = {
    "minlen": "4.0",
    "penwidth":"3.0",
    "concentrate": "true"
}

step_attr = {
    "fontsize": "6",
    "compund": "True",
    "splines":"spline",
}


with Diagram("Orchestration Flow", show=False, graph_attr=graph_attr, node_attr=node_attr, edge_attr=edge_attr ) as diag_clus_orchestration:
    
    with Cluster("SageMaker Sudio", graph_attr=cluster_attr):
        steps= [Sagemaker("SageMaker \nProcessing\n Step"),
                Sagemaker("SageMaker \nHyperParameterTuning\n Step"),
                Sagemaker("SageMaker \nProcessing\n Step"),
                Sagemaker("SageMaker \ncondition\n Step"),
                Sagemaker("SageMaker \nRegisterModel\n Step"),
                Sagemaker("SageMaker \nCreateModel\n Step"),
                Sagemaker("SageMaker \nTransform \n Step"),
                Sagemaker("SageMaker \nProcessing\n Step"),
                Sagemaker("SageMaker \nProcessing\n Step")
        ]
        
         
       

    steps[0] >> steps[1] >> steps[2] >> steps[3] >> steps[4] >> steps[5] >> steps[6] >> steps[7] >> steps[8]
    
diag_clus_orchestration

In [ ]:
from diagrams import Cluster, Diagram, Edge
from diagrams.onprem.analytics import Spark
from diagrams.onprem.compute import Server
from diagrams.onprem.database import PostgreSQL
from diagrams.onprem.inmemory import Redis
from diagrams.onprem.aggregator import Fluentd
from diagrams.onprem.monitoring import Grafana, Prometheus
from diagrams.onprem.network import Nginx
from diagrams.onprem.queue import Kafka

with Diagram(name="Advanced Web Service with On-Premise (colored)", show=False) as dg:
    ingress = Nginx("ingress")

    metrics = Prometheus("metric")
    metrics << Edge(color="firebrick", style="dashed") << Grafana("monitoring")

    with Cluster("Service Cluster"):
        grpcsvc = [
            Server("grpc1"),
            Server("grpc2"),
            Server("grpc3")]

    with Cluster("Sessions HA"):
        primary = Redis("session")
        primary - Edge(color="brown", style="dashed") - Redis("replica") << Edge(label="collect") << metrics
        grpcsvc >> Edge(color="brown") >> primary

    with Cluster("Database HA"):
        primary = PostgreSQL("users")
        primary - Edge(color="brown", style="dotted") - PostgreSQL("replica") << Edge(label="collect") << metrics
        grpcsvc >> Edge(color="black") >> primary

    aggregator = Fluentd("logging")
    aggregator >> Edge(label="parse") >> Kafka("stream") >> Edge(color="black", style="bold") >> Spark("analytics")